<a href="https://colab.research.google.com/github/woshimajintao/BDM-Project/blob/main/Project%202/BDM_P2_Formatted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install delta-spark
!pip install statsmodels
!pip install mlflow

In [ ]:
import mlflow
import mlflow.spark
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.feature import StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, avg, count, when, datediff, current_date, to_date, regexp_replace, month, dayofmonth, sum as _sum, coalesce, lit, isnan, unix_timestamp, lag, from_json, lead, rand, percentile_approx
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, LongType, StringType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import Row
from delta.tables import DeltaTable
from delta import configure_spark_with_delta_pip
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import json
import time
import glob
import pandas as pd
import numpy as np
import tempfile
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Initialize a Spark session
builder = SparkSession.builder \
    .appName("DataProcessingWithDeltaLake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

delta_extensions = spark.conf.get("spark.sql.extensions", "")
print("Delta Lake loaded:", "io.delta.sql.DeltaSparkSessionExtension" in delta_extensions)

Delta Lake loaded: True


# Formatter

## Process Airport Data

Data Ingestion

In [ ]:
file_path = "/content/drive/MyDrive/BDM/Data/Landing_Zone/2023_TransitAeri_FlightRadar_Ppal_Pais.csv"
airport_df = spark.read.options(inferSchema=True, header=True).csv(file_path)
airport_df.show()

+---------------+---------+--------------------+-----------+
|Data_Referencia|Codi_Pais|            Nom_Pais|Nombre_Vols|
+---------------+---------+--------------------+-----------+
|     2023-01-01|        1|             Espanya|        193|
|     2023-01-01|        2|              Itàlia|         63|
|     2023-01-01|        3|          Regne Unit|         57|
|     2023-01-01|        4|              França|         63|
|     2023-01-01|        5|            Alemanya|         50|
|     2023-01-01|        6|            Portugal|         26|
|     2023-01-01|        7|       Països Baixos|         28|
|     2023-01-01|        8|              Suïssa|         26|
|     2023-01-01|        9|             Bèlgica|         10|
|     2023-01-01|       10|          Marroc, el|         21|
|     2023-01-01|       11|   Estats Units, els|          9|
|     2023-01-01|       12|             Àustria|          5|
|     2023-01-01|       13|             Irlanda|         12|
|     2023-01-01|       

Data Cleaning

In [ ]:
# Drop duplicates
airport_df_cleaned = airport_df.dropDuplicates()

# Handle missing values
airport_df_cleaned = airport_df_cleaned.dropna()

airport_df_cleaned.show()
airport_df_cleaned.printSchema()

+---------------+---------+--------------------+-----------+
|Data_Referencia|Codi_Pais|            Nom_Pais|Nombre_Vols|
+---------------+---------+--------------------+-----------+
|     2023-01-02|        6|            Portugal|         29|
|     2023-01-30|        7|       Països Baixos|         20|
|     2023-02-21|        3|          Regne Unit|         58|
|     2023-02-22|       13|             Irlanda|          8|
|     2023-03-03|       23|               Qatar|          4|
|     2023-03-09|        2|              Itàlia|         75|
|     2023-03-16|        2|              Itàlia|         74|
|     2023-03-16|       23|               Qatar|          4|
|     2023-03-17|        7|       Països Baixos|         28|
|     2023-03-28|        5|            Alemanya|         65|
|     2023-03-31|        6|            Portugal|         40|
|     2023-04-02|       20|Emirats Àrabs Uni...|          6|
|     2023-04-03|        9|             Bèlgica|         22|
|     2023-04-18|       

Data Transformation

In [ ]:
# Calculate total tourists per country and find top 5 countries
country_tourists = airport_df_cleaned.groupBy("Nom_Pais").agg(_sum("Nombre_Vols").alias("total_tourists"))
top_countries = country_tourists.orderBy(col("total_tourists").desc()).limit(5)
top_country_list = [row['Nom_Pais'] for row in top_countries.collect()]

# Filter and pivot the DataFrame for the top 5 countries
pivot_df = airport_df_cleaned.filter(col("Nom_Pais").isin(top_country_list)).groupBy("Data_Referencia").pivot("Nom_Pais").agg(_sum("Nombre_Vols"))

# Add a new column for total tourists and fill NaNs with zeros
for country in top_country_list:
    pivot_df = pivot_df.withColumn(country, coalesce(col(country), lit(0)))

pivot_df = pivot_df.withColumn("total_tourists", sum(col(country) for country in top_country_list))

airport_df_final = pivot_df.orderBy("Data_Referencia")

column_renames = {
    "Data_Referencia": "Date",
    "Alemanya": "Germany_Tourists",
    "Espanya": "Spain_Tourists",
    "França": "France_Tourists",
    "Itàlia": "Italy_Tourists",
    "Regne Unit": "UK_Tourists",
    "total_tourists": "Total_Tourists"
}

for old_col, new_col in column_renames.items():
    airport_df_final = airport_df_final.withColumnRenamed(old_col, new_col)

airport_df_final.show(5)

+----------+----------------+--------------+---------------+--------------+-----------+--------------+
|      Date|Germany_Tourists|Spain_Tourists|France_Tourists|Italy_Tourists|UK_Tourists|Total_Tourists|
+----------+----------------+--------------+---------------+--------------+-----------+--------------+
|2023-01-01|              50|           193|             63|            63|         57|           426|
|2023-01-02|              56|           212|             67|            82|         73|           490|
|2023-01-03|              54|           227|             57|            80|         59|           477|
|2023-01-04|              57|           217|             54|            71|         62|           461|
|2023-01-05|              53|           195|             55|            79|         55|           437|
+----------+----------------+--------------+---------------+--------------+-----------+--------------+
only showing top 5 rows



## Process Listing Data

Data Ingestion

In [ ]:
file_path = "/content/drive/MyDrive/BDM/Data/Landing_Zone/listings.csv"
listings_df = spark.read.csv(file_path, header=True, inferSchema=True)
listings_df.show(5)

+------+--------------------+-------+--------------+-------------------+--------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
|    id|                name|host_id|     host_name|neighbourhood_group|       neighbourhood|         latitude|        longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|    license|
+------+--------------------+-------+--------------+-------------------+--------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
| 17475|Rental unit in 08...|  65623|          Luca|           Eixample|la Dreta de l'Eix...|         41.3993

Data Cleaning

In [ ]:
listings_df_cleaned = listings_df.dropDuplicates().dropna()

Data Transformation

In [ ]:
listings_df_cleaned = listings_df_cleaned.fillna({'license': 'Unknown', 'reviews_per_month': 0})

# Convert price to integer
listings_df_cleaned = listings_df_cleaned.withColumn("price", col("price").cast("integer"))

# Calculate average price per neighbourhood
avg_price_per_neighbourhood = listings_df_cleaned.groupBy("neighbourhood").agg(avg("price").alias("avg_price"))

# Calculate number of listings per neighbourhood
count_listings_per_neighbourhood = listings_df_cleaned.groupBy("neighbourhood").agg(count("id").alias("listings_count"))

# Handle date columns
listings_df_cleaned = listings_df_cleaned.withColumn("last_review", col("last_review").cast("date"))
listings_df_cleaned = listings_df_cleaned.withColumn("days_since_last_review", datediff(current_date(), col("last_review")))

listings_df_final = listings_df_cleaned.withColumn(
    "room_type_encoded",
    when(col("room_type") == "Entire home/apt", 1)
    .when(col("room_type") == "Private room", 2)
    .when(col("room_type") == "Shared room", 3)
    .when(col("room_type") == "Hotel room", 4)
    .otherwise(0)
)

listings_df_final = listings_df_final.drop("price")
listings_df_final = listings_df_final.drop("minimum_nights")
listings_df_final = listings_df_final.drop("maximum_nights")

listings_df_final.show(5)

+------+--------------------+-------+----------+-------------------+--------------------+--------+---------+---------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-------------+----------------------+-----------------+
|    id|                name|host_id| host_name|neighbourhood_group|       neighbourhood|latitude|longitude|      room_type|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|      license|days_since_last_review|room_type_encoded|
+------+--------------------+-------+----------+-------------------+--------------------+--------+---------+---------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-------------+----------------------+-----------------+
|206167|Rental unit in Ba...|1014050|   Minerva|     Sants-Montjuïc|        el Poble Sec| 41.3724|   2.164

## Process Calendar Data

Data Ingestion

In [ ]:
file_path = "/content/drive/MyDrive/BDM/Data/Landing_Zone/calendar.csv"
calendar_df = spark.read.csv(file_path, header=True, inferSchema=True)
calendar_df.show(5)

+----------+----------+---------+-------+--------------+--------------+--------------+
|listing_id|      date|available|  price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+-------+--------------+--------------+--------------+
|    198958|2023-12-14|        t|$190.00|          NULL|             2|           365|
|    198958|2023-12-15|        t|$190.00|          NULL|             2|           365|
|    198958|2023-12-16|        t|$190.00|          NULL|             2|           365|
|    198958|2023-12-17|        t|$190.00|          NULL|             2|           365|
|    198958|2023-12-18|        t|$190.00|          NULL|             2|           365|
+----------+----------+---------+-------+--------------+--------------+--------------+
only showing top 5 rows



Data Cleaning

In [ ]:
calendar_df_cleaned = calendar_df.dropDuplicates()
calendar_df_cleaned.cache()

# Remove currency symbols and commas from the price column, then convert to float
calendar_df_cleaned = calendar_df_cleaned.withColumn("price", regexp_replace(col("price"), "[$,]", "").cast("float"))
calendar_df_cleaned = calendar_df_cleaned.dropna(subset=["price"])

avg_price = calendar_df_cleaned.agg(avg("price")).first()[0]

# Handle missing values by filling missing prices with the average price
calendar_df_cleaned = calendar_df_cleaned.na.fill({'price': avg_price})

Data Transformation

In [ ]:
calendar_df_cleaned = calendar_df_cleaned.withColumn("date", to_date(col("date"), 'yyyy-MM-dd'))

# Calculate availability percentage per listing_id
availability_df = calendar_df_cleaned.groupBy("listing_id").agg(
    (count(when(col("available") == "t", 1)) / count("available") * 100).alias("availability_percentage")
)

# Join availability_df back to the main dataframe
calendar_df_final = calendar_df_cleaned.join(availability_df, on="listing_id", how="left")

# Show the transformed DataFrame
calendar_df_final.show(5)

+----------+----------+---------+------+--------------+--------------+--------------+-----------------------+
|listing_id|      date|available| price|adjusted_price|minimum_nights|maximum_nights|availability_percentage|
+----------+----------+---------+------+--------------+--------------+--------------+-----------------------+
|     36763|2024-05-12|        t|  30.0|          NULL|            31|            65|     57.534246575342465|
|     36763|2024-02-07|        f|  30.0|          NULL|            31|            65|     57.534246575342465|
|     40983|2024-11-20|        t| 205.0|          NULL|             2|           364|      78.63013698630137|
|     46153|2024-03-03|        f|  70.0|          NULL|            31|          1125|      66.57534246575342|
|  20472746|2024-11-28|        t|1292.0|     $1,292.00|             4|             7|      88.76712328767124|
+----------+----------+---------+------+--------------+--------------+--------------+-----------------------+
only showi

## Data Integration

In [ ]:
# Merge Listings and Calendar
merged_df = listings_df_final.join(calendar_df_final, listings_df_final.id == calendar_df_final.listing_id, "inner")
merged_df = merged_df.drop(calendar_df_final.listing_id)
merged_df.show(5)

+-----+--------------------+-------+---------+-------------------+--------------------+--------+---------+---------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+----------------------+-----------------+----------+---------+-----+--------------+--------------+--------------+-----------------------+
|   id|                name|host_id|host_name|neighbourhood_group|       neighbourhood|latitude|longitude|      room_type|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|    license|days_since_last_review|room_type_encoded|      date|available|price|adjusted_price|minimum_nights|maximum_nights|availability_percentage|
+-----+--------------------+-------+---------+-------------------+--------------------+--------+---------+---------------+-----------------+-----------+-----------------+------------------------------+----------------+--

In [ ]:
# Merge with Airport Data
merged_df = merged_df.withColumn("month_day", col("date").substr(6, 5))
airport_df_final = airport_df_final.withColumn("month_day", col("Date").substr(6, 5))
final_merged_df = merged_df.join(airport_df_final.drop("Date"), on="month_day", how="inner")
final_merged_df = final_merged_df.drop("month_day")
final_merged_df = final_merged_df.drop("adjusted_price")

columns_to_drop = [col for col in final_merged_df.columns if col.endswith("price") and col != "price"]
final_merged_df = final_merged_df.drop(*columns_to_drop)

final_merged_df.show(5)

+-----+--------------------+-------+---------+-------------------+--------------------+--------+---------+---------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+----------------------+-----------------+----------+---------+-----+--------------+--------------+-----------------------+----------------+--------------+---------------+--------------+-----------+--------------+
|   id|                name|host_id|host_name|neighbourhood_group|       neighbourhood|latitude|longitude|      room_type|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|    license|days_since_last_review|room_type_encoded|      date|available|price|minimum_nights|maximum_nights|availability_percentage|Germany_Tourists|Spain_Tourists|France_Tourists|Italy_Tourists|UK_Tourists|Total_Tourists|
+-----+--------------------+-------+---------+-------------------+----

In [ ]:
final_merged_df.columns

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'number_of_reviews_ltm',
 'license',
 'days_since_last_review',
 'room_type_encoded',
 'date',
 'available',
 'price',
 'minimum_nights',
 'maximum_nights',
 'availability_percentage',
 'Germany_Tourists',
 'Spain_Tourists',
 'France_Tourists',
 'Italy_Tourists',
 'UK_Tourists',
 'Total_Tourists']

## Data Filtering

In [ ]:
filtered_df = final_merged_df.filter(col("price") >= 0)
filtered_df = filtered_df.filter((col("date") >= "2020-01-01") & (col("date") <= "2024-06-15"))
filtered_df = filtered_df.dropna(subset=["price", "availability_percentage"])
filtered_df.show(5)

+-----+--------------------+-------+---------+-------------------+--------------------+--------+---------+---------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+----------------------+-----------------+----------+---------+-----+--------------+--------------+-----------------------+----------------+--------------+---------------+--------------+-----------+--------------+
|   id|                name|host_id|host_name|neighbourhood_group|       neighbourhood|latitude|longitude|      room_type|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|    license|days_since_last_review|room_type_encoded|      date|available|price|minimum_nights|maximum_nights|availability_percentage|Germany_Tourists|Spain_Tourists|France_Tourists|Italy_Tourists|UK_Tourists|Total_Tourists|
+-----+--------------------+-------+---------+-------------------+----

In [ ]:
row_count = filtered_df.count()
column_count = len(filtered_df.columns)

print(f"Number of rows after filtering: {row_count}")
print(f"Number of columns: {column_count}")

Number of rows after filtering: 1852749
Number of columns: 30


## Data Validation

In [ ]:
# Check for duplicate rows
duplicate_count = filtered_df.count() - filtered_df.dropDuplicates().count()
print(f"Number of duplicate rows: {duplicate_count}")

# Separate numeric and non-numeric columns
numeric_columns = [field.name for field in filtered_df.schema.fields if field.dataType in ['IntegerType', 'LongType', 'FloatType', 'DoubleType']]
non_numeric_columns = [field.name for field in filtered_df.schema.fields if field.dataType not in ['IntegerType', 'LongType', 'FloatType', 'DoubleType']]

# Check for null and missing values in numeric columns
null_counts_numeric = filtered_df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in numeric_columns])
null_counts_numeric.show()

# Check for null values in non-numeric columns
null_counts_non_numeric = filtered_df.select([count(when(col(c).isNull(), c)).alias(c) for c in non_numeric_columns])
null_counts_non_numeric.show()

# Verify data types
filtered_df.printSchema()

# Price should be non-negative
invalid_prices = filtered_df.filter(col("price") < 0).count()
print(f"Number of invalid price values: {invalid_prices}")

# Availability percentage should be between 0 and 100
invalid_availability = filtered_df.filter((col("availability_percentage") < 0) | (col("availability_percentage") > 100)).count()
print(f"Number of invalid availability_percentage values: {invalid_availability}")

# Uniqueness check for IDs
unique_id_count = filtered_df.select("id").distinct().count()
total_id_count = filtered_df.count()
print(f"Number of unique ids: {unique_id_count}, Total ids: {total_id_count}")

# Latitude should be between -90 and 90
invalid_latitude = filtered_df.filter((col("latitude") < -90) | (col("latitude") > 90)).count()
print(f"Number of invalid latitude values: {invalid_latitude}")

# Longitude should be between -180 and 180
invalid_longitude = filtered_df.filter((col("longitude") < -180) | (col("longitude") > 180)).count()
print(f"Number of invalid longitude values: {invalid_longitude}")

# Summary statistics
filtered_df.describe(["price", "minimum_nights", "number_of_reviews", "reviews_per_month", "availability_365", "availability_percentage"]).show()

# Distribution checks for specific columns
filtered_df.groupBy("room_type").count().show()
filtered_df.groupBy("neighbourhood_group").count().show()

# Consistency checks : Last review should be before the current date
invalid_last_review = filtered_df.filter(col("last_review") > current_date()).count()
print(f"Number of invalid last_review values: {invalid_last_review}")

Number of duplicate rows: 0
++
||
++
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
++
only showing top 20 rows

+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-------+----------------------+-----------------+----+---------+-----+--------------+--------------+-----------------------+----------------+--------------+---------------+--------------+-----------+--------------+
| id|name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|license|days_since_last_review|room_type_encoded|date|available|price|minimum_nights|maximum_nights|availability_percentage|Germany_Tourists|Spain_Tourists|France_Tourists|Italy_Tourists|UK_Tourists|Total_Tourists|
+---+----+-------+--------

## Store to Formatted Zone

In [ ]:
delta_lake_path = "/content/drive/MyDrive/BDM/Data/Formatted_Zone/filtered_df"

filtered_df.write.format("delta").option("overwriteSchema", "true").mode("overwrite").save(delta_lake_path)